In [1]:
#-------------------------------------------------------------------------------
# Los datos de Datawarehouse(DWH) se bajaron del Google Cloud Platform a DRIVE
# se preparo la data para ser leida por el modelo y solo contiene los 
# registros VUL(Vulnerabilidad), datos calculados con otras tablas que estan 
# en el DWH. Se monta el DRIVE para leer la data
#-------------------------------------------------------------------------------
# OBJETIVO: Generar un archivo .csv en donde el modelo de machine learning 
# en base a la informacion del 2010 hasta el 2020, va a predecir el 
# indice de vulnerabilidad del año 2021, usando el modelo de regresion lineal
#-------------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#-------------------------------------------------------------------------------
# codigo machine learning modelo de regresion predictiva
# predecir año 2021 con el indice de vulnerabilidad, tomando como data
# los años del 2010 al 2020 que estan en la web del banco mundial
#-------------------------------------------------------------------------------

# cargamos las librerias a utilizar
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# hacemos la ingesta de la data de remesas por paises
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MLparaPFbootcampDS/dwhMLVULprepDATA.csv')


# Seleccionar país para la predicción
dfpaises = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MLparaPFbootcampDS/tablapaises.csv')
#paises= ['ARG', 'PER', 'COL']
paises = dfpaises['codigopais']
indice= 1
paispredecir = paises[indice]

# creamos un dataframe copia para añadir la nueva columna de prediccion
# y grabar la prediccion que haga el machine learning
df_ml = df.copy()
df_ml['prediccion']= 0

# Convertir de ancho a largo
df_melted = pd.melt(df, id_vars=['codigopais'], var_name='year', value_name='remittances')

# eliminamos valores NaN
df_melted.dropna(inplace=True)

# Convertir year a variable numérica
df_melted['year'] = pd.to_numeric(df_melted['year'])

# Convertir remittances a tipo float
#df_melted['remittances'] = df_melted['remittances'].str.replace(',', '.').astype(float)

# Convertir codigopais a variable numérica mediante LabelEncoder
le = LabelEncoder()
df_melted['codigopais'] = le.fit_transform(df_melted['codigopais'])



# hacemos un for por todos los paises
for indice in range(len(paises)):
  paispredecir = paises[indice]
   
  # Filtrar datos por país seleccionado
  df_pais = df_melted[df_melted['codigopais'] == le.transform([paispredecir])[0]]

  # Ajustar modelo de regresión
  X = df_pais[df_pais['year'] <= 2020][['year', 'codigopais']]
  y = df_pais[df_pais['year'] <= 2020]['remittances']
  reg = LinearRegression().fit(X, y)

  # Hacer predicciones para el país seleccionado
  añopredecir = 2021
  X_pred = np.array([[añopredecir, le.transform([paispredecir])[0]]])
  y_pred = reg.predict(X_pred)
  print(f"Predicción para {paispredecir} en {añopredecir}: {y_pred[0]:.4f}")

  df_ml.loc[df_ml['codigopais'] == paispredecir,'prediccion'] = round(y_pred[0], 4)


# al finalizar el for, guarda el nuevo .csv del dataframe: df_ml
df_ml.to_csv('/content/drive/MyDrive/Colab Notebooks/MLparaPFbootcampDS/dwhMLVULprepDATAPREDICCIONML.csv', index = False)
# este archivo csv sera utilizado en el dashboard en looker en el GCP, donde se encuentra el DWH



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature n

Predicción para ARG en 2021: 1.9532
Predicción para BOL en 2021: 4.3838
Predicción para BRA en 2021: 23.8985
Predicción para CAN en 2021: 6.7980
Predicción para CHL en 2021: -4.0605
Predicción para COL en 2021: 932.8856
Predicción para CRI en 2021: 0.2093
Predicción para CUB en 2021: 89.6083
Predicción para ECU en 2021: 18.9817
Predicción para SLV en 2021: 95.6165
Predicción para USA en 2021: 4.2799
Predicción para GTM en 2021: 108.1364
Predicción para GUY en 2021: 1.9219
Predicción para HTI en 2021: -158.2025
Predicción para HND en 2021: 282.4469
Predicción para JAM en 2021: 1.2018
Predicción para MEX en 2021: 39.5127
Predicción para NIC en 2021: 51.3108
Predicción para PAN en 2021: 0.5382
Predicción para PRY en 2021: 9.3537
Predicción para PER en 2021: -2.6788
Predicción para DOM en 2021: 8.0669
Predicción para URY en 2021: 0.8097
Predicción para VEN en 2021: 168740.8758


In [5]:
df_comprobar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MLparaPFbootcampDS/dwhMLVULprepDATAPREDICCIONML.csv')


In [6]:
df_comprobar

,codigopais,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,prediccion
0,ATG,0.0227,0.0195,0.0153,0.0118,0.0078,0.0041,0.0010,0.0905,0.0001,0.0001,0.0001,0.0000
1,ARG,0.5264,0.0866,0.0948,0.1288,2.6466,2.4740,0.6901,1.8830,1.2626,2.8112,0.2219,1.9532
2,BHS,0.0712,0.0536,0.0381,0.0368,0.0354,0.0621,0.0833,0.0185,0.0248,0.2967,0.0105,0.0000
3,BRB,0.0023,0.0035,0.0040,0.0039,0.0039,0.0038,0.0091,0.0038,0.0037,0.0147,0.0001,0.0000
4,BLZ,0.3074,0.2913,0.2743,0.2352,0.2422,0.1793,0.3630,0.2026,0.1961,0.1878,1.1925,0.0000
5,BOL,60.2055,11.0158,8.4380,5.9578,70.3912,7.6435,5.8587,3.9913,3.4571,25.7134,10.2342,4.3838
6,BRA,19.9493,10.9756,0.7226,4.0674,11.0435,6.7911,8.6466,1.5162,2.0437,26.6460,48.7105,23.8985
7,CAN,4.8730,4.5093,4.7281,2.3441,3.7968,5.0901,5.4594,6.2102,8.7701,7.5710,3.9461,6.7980
8,CHL,156.7839,1.7045,1.5796,2.1168,66.5806,76.2133,8.2895,14.0166,15.2293,15.7494,7.4602,-4.0605
9,COL,1361.1072,780.5439,740.0890,732.4043,790.0961,1102.4986,1333.8550,1012.0615,833.9237,861.1853,943.2533,932.8856
